In [121]:
using Flux, DataFrames, CSV, ProgressMeter, DiffEqFlux, LinearAlgebra, DifferentialEquations

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1317


In [89]:
data = CSV.File(joinpath(pwd(), "..", "..", "..", "data", "measures.csv")) |> DataFrame
@. data[!, :i_norm] = sqrt(data.i_d^2 + data.i_q^2);

In [94]:
target_cols = ["pm", "stator_tooth", "stator_winding", "stator_yoke"]
gdf = groupby(data, :profile_id)

# just work on profile 60 for now (dummy)
p6 = get(gdf, (profile_id = 60,), nothing);

In [154]:
#lptn_1d!([0, 0, 0, 0], u0, p, 1)
[size(s) for s in p]


3-element Vector{Tuple{Int64}}:
 (15,)
 (4,)
 (4,)

In [168]:
n_temps = 6

arranged_p6 = Array(p6[:, [target_cols..., ["ambient", "coolant"]...]]) ./ 100
i_norm = Array(p6.i_norm) ./ maximum(p6.i_norm)

# populate adjacency matrix
function get_adj_mat(n_temps_)
    adj_mat = zeros(Int8, n_temps_, n_temps_)
    k = 1
    for col_j = 1:n_temps_
        for row_i = col_j+1:n_temps_
            adj_mat[row_i, col_j] = k
            k += 1
        end
    end
    adj_mat = adj_mat + adj_mat'
end

n_conds = Int(0.5 * n_temps * (n_temps - 1))

adj_mat = get_adj_mat(n_temps)

function lptn_1d!(du, u, p, t)
    ex_idx = clamp(Int(round(t)), 1, nrow(p6))
    conductances, loss_p, caps = p
    current_temps = arranged_p6[ex_idx, :]
    temp_diffs =
        [
            sum(
                current_temps[j] - u[i] * conductances[adj_mat[i, j]] for
                j in 1:n_temps if j != i
            ) for i = 1:size(target_cols, 1)
        ]
    du[:] .= caps .* (i_norm[ex_idx] .* loss_p .+ temp_diffs)

end

# Initial condition
u0 = Array(p6[1, target_cols]) ./ 100

# Simulation interval and intermediary points
tspan = (0.0, nrow(p6))
tsteps = 0:1:nrow(p6)

# LV equation parameter. p = [conductances, loss_factors, inverse_capacitances]
p = [
    abs.(randn(n_conds)),
    abs.(randn(size(target_cols, 1))),
    10 .^ (-1 .- 2 .* rand(Float64, size(target_cols, 1))),
]

# Setup the ODE problem, then solve
prob = ODEProblem(lptn_1d!, u0, tspan, p)
sol = solve(prob)


retcode: Success
Interpolation: automatic order switching interpolation
t: 2368-element Vector{Float64}:
     0.0
     0.16726295353037612
     1.2520705082926606
     3.160298243596209
     4.407748059389485
     6.079330130612717
     8.500169272518395
     9.96149559299328
    12.37392585784253
    15.430504827911736
    17.969698303593283
    20.85875537098152
    25.746205859914678
     ⋮
 14415.052243994642
 14425.135530138608
 14436.16861745069
 14447.7018645112
 14459.109186685178
 14471.550247665264
 14483.118039372966
 14499.792135138752
 14513.612956266681
 14527.23905073592
 14541.437744474642
 14543.0
u: 2368-element Vector{Vector{Float64}}:
 [0.35525391114555227, 0.37866300008276776, 0.38895034995156424, 0.3801852308028428]
 [0.3593384685421805, 0.38335861160294765, 0.39088846414007145, 0.38713898670161456]
 [0.3843894871794353, 0.4102337586197144, 0.40196098353550996, 0.4308014739333146]
 [0.4266747014875732, 0.4527733358159091, 0.421488055593011, 0.5031175407535396]
 [0

In [171]:
using Plots
dfui = p6[!, target_cols]
plot(
    Array(dfui) ./ 100,
    lab = reshape(vec(target_cols), (1, :)),
    line=(2, reshape([:dash, :dash, :dash, :dash], 1, :)),
    xaxis = "Sample",
    yaxis = "Temperature in x100°C",
)
plot!(sol)


